31.05.2022

# Referat: Einführung in spaCy

Sommersemester 2023

Kurs: DH1 - Sprache & Text

Dozent: Andreas Blombach

Gehalten von: Christian Hagenest

# Inhalt

- Was ist spaCy?
- Installation
- Features
    - Tokenization
    - Linguistische Annotationen
    - Named Entity Recognition
    - Wortvektoren & Ähnlichkeit
    - Sentimentanalyse

- Vergleich mit anderen Libraries


# Was ist spaCy?

- Python Library für NLP (Natural Language Processing)
- Fokus auf einfach Benutzbarkeit und gute Performance
- Opiniertes Design
    - d.h. SpaCy lässt dem User wenig Freiheiten bei der Wahl von Algorithmen, sondern wählt selbst die "besten" Algorithmen aus
    - Zielgruppe: Mehr Industrie als Forschung
- Open Source (MIT Lizenz)


# Installation

### Mit Pip (empfohlen)
Hier mit den kleinen, schnellen Pipelines (Je ca. 50 MB)


```bash
pip install -U pip setuptools wheel
pip install -U spacy
python -m spacy download en_core_web_sm
python -m spacy download de_core_news_sm
```

### Mit Conda
Hier mit den größeren, genaueren Pipelines (Je ca. 500 MB)

```bash
conda install -c conda-forge spacy
python -m spacy download en_core_web_trf
python -m spacy download de_core_news_lg
```
*'-c conda-forge'* solltet ihr nicht benötigen, wenn ihr statt Anaconda bereits CondaForge oder MiniForge verwendet.

### Am Besten: Interaktive Anleitung auf spacy.io
[Install spaCy](https://spacy.io/usage)

![resources/install.png](resources/install.png)

Wenn ihr das Notebook selbst ausführen möchtet, braucht ihr auch die folgenden Libraries:

```
pandas plotly numpy spacyfishing spacytextblob textblob-de
```

In [400]:
# Datenquelle

import pandas as pd

df = pd.read_json("data/taz.json")
example = df['description'][3]
print("Beispiel-Text: \n\n", example, "\n\n")

# Erste Schritte mit spaCy

import spacy

nlp = spacy.load('de_core_news_lg')

Beispiel-Text: 

 Die Polizei auf den Bahamas hat den Gründer und Ex-Geschäftsführer der Kryptoplattform FTX verhaftet. Die US-Regierung veranlasste die Festnahme. 




# Features

## Tokenization

- Spacy kann Wörter und Satzzeichen erkennen und in Token umwandeln
- Erkennt z.B. auch, dass "z.B." ein Token ist

In [401]:
doc = nlp(example)
print("Tokenization: \n\n", [token.text for token in doc], "\n\n")

Tokenization: 

 ['Die', 'Polizei', 'auf', 'den', 'Bahamas', 'hat', 'den', 'Gründer', 'und', 'Ex-Geschäftsführer', 'der', 'Kryptoplattform', 'FTX', 'verhaftet', '.', 'Die', 'US-Regierung', 'veranlasste', 'die', 'Festnahme', '.'] 




## Linguistische Annotationen

- Spacy kann eine Vielzahl von linguistischen Annotationen automatisch vornehmen
    - z.B. Lemma - Grundform des Wortes
    - POS - Part of Speech (Wortart)
    - Tag - Feinere Unterscheidung der Wortart
    - Dep - Abhängigkeiten zwischen Wörtern
    - Shape - Die Form eines Wortes, z.B. Groß-/Kleinschreibung, Zeichensetzung, Länge
    - isAlpha - Ist das Token ein Wort?
    - isStop - Ist das Token ein Stoppwort? (z.B. "der", "die", "das")    
- Hierfür benötigt Spacy ein vortrainiertes Modell


In [402]:
annotated = [[token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop] for token in doc]

annotated = pd.DataFrame(annotated, columns=["Text", "Lemma", "POS", "Tag", "Dep", "Shape", "IsAlpha", "IsStop"])
annotated

,Text,Lemma,POS,Tag,Dep,Shape,IsAlpha,IsStop
0,Die,der,DET,ART,nk,Xxx,True,True
1,Polizei,Polizei,NOUN,NN,sb,Xxxxx,True,False
2,auf,auf,ADP,APPR,mnr,xxx,True,True
3,den,der,DET,ART,nk,xxx,True,True
4,Bahamas,Bahamas,PROPN,NE,nk,Xxxxx,True,False
5,hat,haben,AUX,VAFIN,ROOT,xxx,True,True
6,den,der,DET,ART,nk,xxx,True,True
7,Gründer,Gründer,NOUN,NN,oa,Xxxxx,True,False
8,und,und,CCONJ,KON,cd,xxx,True,True
9,Ex-Geschäftsführer,Ex-Geschäftsführer,NOUN,NN,cj,Xx-Xxxxx,False,False


In [403]:
# SpaCy kann das Verhältnis zwischen Wörtern in einem Satz visuell darstellen
from spacy import displacy

displacy.render(doc, style="dep", jupyter=True)

## Named Entity Recognition

SpaCy kann Named Entities erkennen, klassifizieren und visualisieren.
Gerade auf deutsch kommt spaCy hierbei manchmal an seine Grenzen, selbst wenn man wie hier das große Modell benutzt:

In [404]:
example_long = df['text'][3]
doc_long = nlp(example_long)


displacy.render(doc_long, style="ent", jupyter=True)

## Linken von Named Entities zu Wikidata mit spacyfishing

### Bugged: Ich bekomme die Pipeline nicht zu laufen. Ich versuche am Dienstag noch zu debuggen, ansonsten stelle ich eine andere Pipeline vor.

Die spaCy-Pipeline spacyfishing kann Named Entities mit automatisch mit Wikidata-Einträgen verlinken.
Leider funktioniert spacyfishing nur mit den englischen Modellen.

Achtung: Die meisten SpaCy-Pipelines sind nicht Teil von Anaconda oder CondaForge und müssen daher mit Pip installiert werden. Es ist möglich mit Conda ein virtuelles Environment anzulegen, dort Pip zu installieren und dann die Pipelines zu installieren. Das ist aber etwas komplizierter - einfacher ist es für alle Abhängigkeiten Pip zu verwenden.

In [405]:
#import spacyfishing
#nlp.add_pipe('entityfisher')
#with open("data/example_en.txt") as f:
#    doc_en = nlp(f.read())
#displacy.render(doc_en, style="ent", jupyter=True)

In [406]:
#fished = [[ent.text, ent.label_, ent._.url_wikidata, ent._.nerd_score] for ent in doc_en.ents]
#fished = pd.DataFrame(fished, columns=["Text", "Label", "Wikidata", "NerdScore"])
#fished

## Ähnlichkeit von Wörtern

- SpaCy kann Wörter als Vektoren darstellen, die die Bedeutung des Wortes repräsentieren. 
- Diese Vektoren können dann verglichen werden, um die Ähnlichkeit zwischen Wörtern zu bestimmen.

- Achtung: Die kleinen Modelle von SpaCy können liefern keine Vektoren mit.

In [407]:
example_words = nlp("Haus Turm Katze arithmetisch yfhrd")

vectors = [[token.text, token.has_vector, token.vector_norm, token.is_oov] for token in example_words]

vectors = pd.DataFrame(vectors)
vectors.columns = ["Text", "HasVector", "VectorNorm", "IsOOV"]
vectors

,Text,HasVector,VectorNorm,IsOOV
0,Haus,True,56.222073,False
1,Turm,True,49.403755,False
2,Katze,True,43.214478,False
3,arithmetisch,False,0.000000,True
4,yfhrd,False,0.000000,True


In [408]:
import plotly.express as px
import numpy as np

similarity = []

for token1 in example_words:
    for token2 in example_words:
        similarity.append(token1.similarity(token2))

tokens = [token.text for token in example_words]
similarity = pd.DataFrame(np.array(similarity).reshape(tokens.__len__(), tokens.__len__()))
similarity.columns = tokens
similarity.index = tokens
# heatmap
fig = px.imshow(similarity)
fig.show()


/tmp/ipykernel_13640/440262561.py:8: UserWarning:

[W008] Evaluating Token.similarity based on empty vectors.



- Achtung: Die Ähnlichkeit von Wörten ist sehr ungenau:
    - Ähnlichkeit ist subjektiv
    - Die Reihenfolge der Wörter hat keine Auswirkungen auf das Ergebnis
    - Bsp: Sind die Sätze "Ich mag Züge" und "Ich mag Züge nicht" ähnlich?

In [409]:
sentence1 = nlp("Ich mag Züge")
sentence2 = nlp("Ich mag Züge nicht")

print("Ähnlichkeit: ", sentence1.similarity(sentence2))

Ähnlichkeit:  0.9622064306893177


# Sentimentanalyse mit spacytextblob

- Mit spacytextblob (und textblobde) kann spaCy Sentimentanalysen durchführen
- Polarity: Wie positiv oder negativ ist der Text?
- Subjectivity: Wie subjektiv ist der Text?

In [410]:
from spacytextblob.spacytextblob import SpacyTextBlob
from textblob_de import TextBlobDE

@spacy.registry.misc("spacytextblob.de_blob")
def create_de_blob():
    return TextBlobDE

nlp.add_pipe("spacytextblob", config={
    "blob_only": True,
    "custom_blob": {"@misc": "spacytextblob.de_blob"}
})

In [411]:
sentiment_texts = [
    "Ich mag das Bild.",
    "Das Bild ist hässlich.",
    "Das Bild ist expressionistisch.",
    "Mir gefällt die Wand besser als das Bild.",
]
sentiments = [[text,
            nlp(text)._.blob.sentiment.polarity,
            nlp(text)._.blob.sentiment.subjectivity,
            ] for text in sentiment_texts]

sentiments = pd.DataFrame(sentiments, columns=["Text", "Polarity", "Subjectivity"])
sentiments


,Text,Polarity,Subjectivity
0,Ich mag das Bild.,1.0,0.0
1,Das Bild ist hässlich.,-1.0,0.0
2,Das Bild ist expressionistisch.,0.0,0.0
3,Mir gefällt die Wand besser als das Bild.,0.0,0.0


In [412]:
# Beispiel: Sentimentanalyse von Nachrichtenartikeln

df["Polarity"] = df["text"].apply(lambda x: nlp(x)._.blob.sentiment.polarity)
df["Subjectivity"] = df["text"].apply(lambda x: nlp(x)._.blob.sentiment.subjectivity)
df[["title", "text", "Polarity", "Subjectivity"]].sort_values(by="Polarity", ascending=False)

,title,text,Polarity,Subjectivity
2,Ex-Basketballer Rodman will zu Putin: Experte ...,Seit 189 Tagen . Alle diplomatischen Bemühung...,0.145333,0.050667
7,Im Wahlkreis von Robert Habeck: Mehr als Minde...,| Es ist keine alltägliche Unterrichtsstund...,0.126905,0.023810
0,Deutsche Meisterschaften im Eiskunstlauf: HipH...,"„Man muss das Leben tanzen“, lautet das Motto ...",0.126316,0.052632
4,Schwangerschaftsabbrüche in Ungarn: Ein Herzkl...,| Ein Herzklopfen soll Abtreibungen in Unga...,0.122321,0.000000
9,Wagner-Söldnertruppe in der Ukraine: Putins Ma...,\n \n Die Rivalität zwischen der...,0.118487,0.057143
5,Verfassungsschutz und Rechtsextremisten: Die B...,Es ist ein Schlag mit Vorlauf. Seit 2019 prüft...,-0.025000,0.083333
6,Verbot von Nakba-Demonstrationen: Palästinense...,"| An diesem Montag ist Tag der Nakba, Arabi...",-0.048889,0.044444
3,Gründer von Crypto-Broker FTX verhaftet: Count...,| Vor einem Monat ging die Kryptowährungsbö...,-0.058824,0.000000
1,Welle der Proteste in Israel: Träumen vom Ende...,| Geld versprach der bedrängte israelische ...,-0.067308,0.166667
8,Buch „Mädchenmeuterei“ von Kirsten Fuchs: In s...,Ist es okay zu lügen? Wenn man damit andere sc...,-0.242793,0.032432


## Benchmark: SpaCy vs andere NLP Libraries

SpaCy hat den Ruf, die schnellste NLP Library zu sein, aber dafür weniger genau zu sein.

